#### 將 Label 好的圖片(圖檔本身、Xml)    
#### 存成TXT檔 和 放入對應的資料夾 生成檔案為 2個 TXT檔(訓練、測試) <== 包含(x,y,w,h,class_id) 
#### 2個資料夾(訓練圖片、測試圖片)

## Include Library

In [1]:
import xml.etree.ElementTree as ET
from os import getcwd

import random
import os
from PIL import Image

### Initial Parameter

In [2]:
wd = getcwd()  # Get Menu Path

fnxml="./readxml"   #readyxml0622    Photo Path
fnimage="./readimg"  #readyimg0622   Xml Path

In [3]:
# Check Path is True
if not os.path.exists(fnxml):
    os.makedirs(fnxml)
if not os.path.exists(fnimage):
    os.makedirs(fnimage)

## Subroutine

In [5]:
# Get file feture parameter for Xml
def convert_annotation_new(image_id, list_file):
    
    # 從檔案載入並解析 XML 資料
    in_file = open(fnxml+'/%s'%(image_id),encoding="utf-8") # 開啟 xml檔
    tree=ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text # 找到困難指數 1的話代表圖片很難學 後續將忽略 

        cls = obj.find('name').text           # 呼叫 xml 檔 "實際標籤"

        # 如果 cls 沒有該類別 或 difficult ==1 忽略
        if cls not in classes or int(difficult)==1:
            continue # 和 break 類似 不同的是 continue 會直接結束這個 for 迴圈  (跳出該檔案)
        cls_id = classes.index(cls)          # 找出該類別 "編號標籤"
        xmlbox = obj.find('bndbox')          # 找到 物體邊界框屬性
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        # 寫入 x,y,w,h,cls_id
        
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))


## Create Dict list

In [6]:
files = os.listdir(fnxml)
filesimage = os.listdir(fnimage)
xmlimage = dict(zip(files,filesimage)) 

for a in range(5):
    print(list(xmlimage.items())[a])

('0.xml', '0.jpg')
('1-1.xml', '1-1.jpg')
('1.xml', '1.jpg')
('10-1.xml', '10-1.jpg')
('10.xml', '10.jpg')


In [12]:
# 方法有點笨
totallen=len(files)
trainlen=int(len(files)*0.7)   #####0.7   分出train_test

In [13]:
ids = list(range(1,totallen+1))  # Create Ini list 
random.shuffle(ids)             # shuffle list
random.shuffle(files)

# ============================ 先切出 訓練的部份 ============================
train = ids[:trainlen]   # Shuffle Seed 
trainxml=files[:trainlen]

trainimage=[]
for x in trainxml:
    # 遞迴出 xml 的 圖檔名稱 Then save to trainimage
    trainimage.append(xmlimage[x])

    
# ============================ 再切出測試部份 ============================
test = ids[trainlen:]   # Shuffle Seed 
testxml=files[trainlen:]

testimage=[]
for x in testxml:
    testimage.append(xmlimage[x])

# ============================ 可視化確認 ============================
print("訓練圖片資料集："," trainimage")
print("訓練標籤資料集："," trainxml")
print("測試圖片資料集："," testimage")
print("測試標籤資料集："," testxml")
print("")
for a in range(10):
    print("Number: "+str(a), end= " ====> ")
    print("圖片名稱: ", end=" : ")
    print('{:<25s}'.format(trainimage[a]),end = "")
    print("標籤名稱: ", end=" : ")
    print('{:<25s}'.format(trainxml[a]))

訓練圖片資料集：  trainimage
訓練標籤資料集：  trainxml
測試圖片資料集：  testimage
測試標籤資料集：  testxml

Number: 0 ====> 圖片名稱:  : 287-1.jpg                標籤名稱:  : 287-1.xml                
Number: 1 ====> 圖片名稱:  : 4-1.jpg                  標籤名稱:  : 4-1.xml                  
Number: 2 ====> 圖片名稱:  : 281.jpg                  標籤名稱:  : 281.xml                  
Number: 3 ====> 圖片名稱:  : 7-1.jpg                  標籤名稱:  : 7-1.xml                  
Number: 4 ====> 圖片名稱:  : 282.jpg                  標籤名稱:  : 282.xml                  
Number: 5 ====> 圖片名稱:  : 286-1.jpg                標籤名稱:  : 286-1.xml                
Number: 6 ====> 圖片名稱:  : 288-1.jpg                標籤名稱:  : 288-1.xml                
Number: 7 ====> 圖片名稱:  : 284.jpg                  標籤名稱:  : 284.xml                  
Number: 8 ====> 圖片名稱:  : 5-1.jpg                  標籤名稱:  : 5-1.xml                  
Number: 9 ====> 圖片名稱:  : 6-1.jpg                  標籤名稱:  : 6-1.xml                  


##  Main Function

In [14]:
#生成的文字檔案
sets=[('raccoon_train'), ('raccoon_test')]  
classes = ["ok","ng","normal"]

# 生成圖片存放資料夾 
path1="./train_testimg"  
path2="./test_testimg"  

if not os.path.exists(path1):
    os.makedirs(path1)
if not os.path.exists(path2):
    os.makedirs(path2)

In [15]:
count=0
for image_set in sets:

    print(image_set)  # 欲建立的 TXT 檔

    list_file = open('./%s_data.txt'%(image_set), 'w')   # 開啟 TXT 檔
    
    if count==0:
        for i in range(len(trainimage)):
            
            # 寫入檔案路徑 (含名稱)
            list_file.write('Raccoon_dataset/images/%s'%(trainimage[i]))
            
            # 將訓練圖片集 存入 PATH1
            img=Image.open(fnimage+'/%s'%(trainimage[i]))
            print(trainimage[i])
            
            img.save(path1+'/%s'%(trainimage[i]))
            
            # 寫入 物件 x,y,w,h,class_id
            convert_annotation_new(trainxml[i], list_file)
            list_file.write('\n')   # 換行
    else:
        # 重複上述 並建立測試資料集
        for i in range(len(testimage)):
            list_file.write('Raccoon_dataset/images/%s'%(testimage[i]))
            img=Image.open(fnimage+'/%s'%(testimage[i]))
            print(trainimage[i])
            img.save(path2+'/%s'%(testimage[i]))
            convert_annotation_new(testxml[i], list_file)
            list_file.write('\n')

    list_file.close()
    count+=1

raccoon_train
287-1.jpg
4-1.jpg
281.jpg
7-1.jpg
282.jpg
286-1.jpg
288-1.jpg
284.jpg
5-1.jpg
6-1.jpg
10-1.jpg
289-1.jpg
8-1.jpg
9-1.jpg
raccoon_test
287-1.jpg
4-1.jpg
281.jpg
7-1.jpg
282.jpg
286-1.jpg
